In [14]:
import os
import pandas as pd
fpRaw = "./data/Environment_Temperature_change_E_All_Data_NOFLAG.csv"

In [50]:
#https://www.kaggle.com/datasets/sevgisarac/temperature-change/
dfRaw = pd.read_csv(fpRaw)
dfRaw.Element.unique()

array(['Temperature change', 'Standard Deviation'], dtype=object)

In [47]:

df = dfRaw
df = df[df.Months == "Meteorological year" ]
df  =df[df.Element == "Temperature change"]

In [53]:
def getColNames(decade):
    years = range(0,10)
    return ["Y"+ str(decade[:-2])+str(year) for year in years]

decades = ["1960s","1970s","1980s","1990s","2000s","2010s"]
df["Y1960"] = 0
for decade in decades:
    years = getColNames(decade)
    df[decade] = df[years].mean(axis=1)
    #.mean(skipna=True,numeric_only=True, axis=0)
df = df[["Area"]+decades]
df.to_json(orient="split")
df.head()

,Area,1960s,1970s,1980s,1990s,2000s,2010s
32,Afghanistan,-0.1294,0.0317,0.2145,0.344700,0.970400,1.1188
66,Albania,0.1005,-0.1760,0.0346,0.396700,0.927000,1.5816
100,Algeria,0.0972,-0.1315,0.5516,0.799800,1.338700,1.5400
134,American Samoa,-0.1020,0.0145,0.3040,0.714667,0.782889,1.1611
168,Andorra,0.0773,-0.1164,0.4487,0.961500,1.199000,1.5096


In [30]:
fileBG = "dataBG.json"

fpBG = os.path.join(os.getcwd(),"data", fileBG)         

file1 = open(completeName, "w")

toFile = raw_input("Write what you want into the field")

file1.write(toFile)

file1.close()

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
32,2,Afghanistan,7020,Meteorological year,7271,Temperature change,�C,-0.080,-0.120,0.882,...,1.564,1.342,0.102,1.237,0.474,1.096,1.473,1.483,1.524,0.893
66,3,Albania,7020,Meteorological year,7271,Temperature change,�C,0.631,0.339,0.084,...,1.294,1.123,1.603,1.495,1.399,1.725,1.697,1.337,2.232,1.911
100,4,Algeria,7020,Meteorological year,7271,Temperature change,�C,0.186,0.101,0.045,...,2.359,1.492,1.238,1.267,1.774,1.224,1.866,1.630,1.326,1.224
134,5,American Samoa,7020,Meteorological year,7271,Temperature change,�C,-0.014,-0.080,0.072,...,1.271,0.790,0.901,1.159,1.086,0.885,1.648,1.268,1.101,1.502
168,6,Andorra,7020,Meteorological year,7271,Temperature change,�C,0.749,0.110,-0.755,...,0.441,1.643,1.233,0.794,1.899,1.619,1.917,1.853,1.826,1.871


In [52]:
[2,3]+["decades"]

[2, 3, 'decades']